## Notebook for Session 9

**Note:** This notebook is to be seen as a 'scratch-pad', illustrating the points made in the slides.

In [1]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_hub as hub
from time import time

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy

### 1. Generation of Some Synthetic Test Data 

Imagine 400 train and 400 test examples with 10 features each. Each example has **two** labels, label 'a' and label 'b'. We construct these examples synthetically by picking 10 random numbers between 0 and 1, and adding for the examples with label $a =  1$ 0.2 to the first five features, and subtracting 0.2 in those features for examples with label $a = 0$. We do the same for label $b$, where we add/subtract 0.1 to features 6-10. (Obviously, this is just a toy example and the details do not matter - we just want to have some data to explore the formalisms.)

In [2]:
train_1_X = np.array([0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_2_X = np.array([0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_3_X = np.array([-0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
train_4_X = np.array([-0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])

train_X = np.concatenate([train_1_X, train_2_X, train_3_X, train_4_X])

test_1_X = np.array([0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_2_X = np.array([0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_3_X = np.array([-0.2] * 5 + [0.1] * 5) + np.reshape(np.random.random(1000),[100,10])
test_4_X = np.array([-0.2] * 5 + [-0.1] * 5) + np.reshape(np.random.random(1000),[100,10])

test_X = np.concatenate([test_1_X, test_2_X, test_3_X, test_4_X])

train_1a_y = train_1b_y = train_2a_y = train_3b_y = np.reshape([[1] * 100], [100,1])
train_2b_y = train_3a_y = train_4a_y = train_4b_y = np.reshape([[0] * 100], [100,1])

test_1a_y = test_1b_y = test_2a_y = test_3b_y = np.reshape([[1] * 100], [100,1])
test_2b_y = test_3a_y = test_4a_y = test_4b_y = np.reshape([[0] * 100], [100,1])

train_a_y = np.concatenate([train_1a_y, train_2a_y, train_3a_y, train_4a_y])
train_b_y = np.concatenate([train_1b_y, train_2b_y, train_3b_y, train_4b_y])

test_a_y = np.concatenate([test_1a_y, test_2a_y, test_3a_y, test_4a_y])
test_b_y = np.concatenate([test_1b_y, test_2b_y, test_3b_y, test_4b_y])


Shapes as expected?

In [3]:
np.shape(train_X)

(400, 10)

In [4]:
train_X[:2,:]

array([[0.73692999, 1.10728784, 1.00251368, 0.48547276, 1.14743858,
        1.0299117 , 0.95879371, 0.40761422, 0.52878452, 0.76707157],
       [0.61915175, 1.00161909, 0.80442121, 1.16581909, 1.17072022,
        0.56239227, 0.66419048, 0.65646977, 0.99728475, 0.95194047]])

In [5]:
np.shape(train_a_y)

(400, 1)

In [6]:
train_a_y[:2, :]

array([[1],
       [1]])

Looks good.

### 2. Keras' Functional API Formalism

Keras' Functional API allows us to construct non-sequential models with multiple inputs, multiple outputs, branched models,  etc..

Key aspects:

* start with definition of inputs through Input layer(s)
* define the layers and outputs as:
$$\rm next\_layer\_activations = layer(layer\_parameters) (previous\_layer\_activations)$$
* define the model the the input(s) and output(s).  
* 'compile' the model by specifying loss function(s), metric(s) and the optimizer.  

We first set things up for one binary classification. It is most convenient to define a function called *build_model()* where we can parametrize aspects we want to vary.

In [7]:
def build_model(activation_function='relu', 
                optimizer='adam', 
                kernel_initializer=tf.keras.initializers.he_normal(seed=1), 
                bias_initializer='zeros'):

    # Define Input layer(s)
    input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

    # Define one hidden layer and act on input
    dense_layer = tf.keras.layers.Dense(10, 
                                        activation=activation_function, 
                                        kernel_initializer=kernel_initializer, 
                                        bias_initializer=bias_initializer,
                                        name='dense') # layer definition
    dense_output = dense_layer(input_numbers)  # layer acting on previous layer's output
    
    # Define one hidden layer and act on input
    dense_layer_2 = tf.keras.layers.Dense(10, 
                                        activation=activation_function, 
                                        kernel_initializer=kernel_initializer, 
                                        bias_initializer=bias_initializer,
                                        name='dense_2') # layer definition
    dense_output_2 = dense_layer_2(dense_output)  # layer acting on previous layer's output

    

    # Define classification layer and act on previous output to classify examples
    classification_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='classification') # layer definition
    classification_output = classification_layer(dense_output_2)   # layer acting on previous layer's output

    # Build and compile model
    model = tf.keras.models.Model(input_numbers, classification_output)
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return model

model = build_model()


model.summary()

W1019 10:25:32.000267 4420785600 deprecation.py:506] From /anaconda3/envs/tf1_14/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1019 10:25:32.031944 4420785600 deprecation.py:323] From /anaconda3/envs/tf1_14/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_numbers (InputLayer)   [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
classification (Dense)       (None, 1)                 11        
Total params: 231
Trainable params: 231
Non-trainable params: 0
_________________________________________________________________


Easy!   

**Question:** Do the dimensions look right? 

### 3.  Training Optimizations

Great, let's do some experimenting. We run the model above for the synthetic data set we defined, only considering the first label, and look at a few variations. Specifically, let's play with:

* the **activation function** in the hidden layer     
* the **initialization**    
* the **optimizer**

We run everything for 40 epochs and compare the results. (We split the training into two parts to cut down on reporting. 'verbose=0' supresses output. But we still want to see the final loss so we add one epoch with 'verbose=1'.)

In [14]:
model_1 = build_model()  # This one should be good...

model_1.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_1.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 55us/sample - loss: 0.1802 - val_loss: 0.2568


This is our supposed best setup, and it defines our baselines. 

Let's veer off and see how loss, etc. change. First, we use $tanh()$ as opposed to $relu()$:

In [15]:
model_2 = build_model(activation_function='tanh')  

model_2.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_2.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 96us/sample - loss: 0.2039 - val_loss: 0.2737


A bit worse. [Of course, the initialization is random, so things may be different on future runs.]

Next, we change the optimizer to stochastic gradient descent:

In [16]:
model_3 = build_model(optimizer='sgd')  

model_3.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_3.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 51us/sample - loss: 0.4382 - val_loss: 0.4778


A lot worse!!

Going back to the activation function, what about $sigmoid()$?

In [17]:
model_4 = build_model(activation_function='sigmoid')  

model_4.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_4.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 92us/sample - loss: 0.4543 - val_loss: 0.4715


Not good!

Lastly, what about choosing general random matrix initialization as opposed to He/Xavier? 

In [18]:
model_5 = build_model(kernel_initializer='random_normal')  

model_5.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=39,
        verbose=0)

model_5.fit(
        train_X,
        train_a_y,
        validation_data=[test_X, test_a_y],
        epochs=1,
        verbose=1)

W1019 10:26:24.996901 4420785600 deprecation.py:506] From /anaconda3/envs/tf1_14/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 99us/sample - loss: 0.1853 - val_loss: 0.2438


A bit worse too.

**Critical Note:** We were cheating a bit here! There is a stochastic component and if one re-reruns the cells results will differ. In fact, *in this simple case* the last configuration is often the best.

### 4. Splitting Models - Multiple Outputs

Now, let's go back to our dataset and let's try to predict two labels. We want to model that by *splitting* the model, i.e., two distinct hidden layers act on the input, and we use those two branches to predict the two labels:  

In [19]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = tf.keras.layers.Dense(10, activation='relu', name='dense_1')
dense_layer_2 = tf.keras.layers.Dense(10, activation='relu', name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)     # hideen layer 1 acting on input
dense_output_2 = dense_layer_2(input_numbers)     # hideen layer 2 acting on input



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')  # classification layer 1 for first branch

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')  # classification layer 2 for second branch


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model = tf.keras.models.Model(model_input, model_output)
model.compile(loss=losses,  optimizer='adam', metrics=metrics)




In [20]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_numbers (InputLayer)      [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           110         input_numbers[0][0]              
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           110         input_numbers[0][0]              
__________________________________________________________________________________________________
classification_1 (Dense)        (None, 1)            11          dense_1[0][0]                    
___________________________________________________________________________________________

In [21]:
model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=150,
    verbose=1
)

Train on 400 samples, validate on 400 samples
Epoch 1/150
400/400 [==============================] - 0s 1ms/sample - loss: 1.4410 - classification_1_loss: 0.7315 - classification_2_loss: 0.7043 - classification_1_acc: 0.5200 - classification_2_acc: 0.4900 - val_loss: 1.4202 - val_classification_1_loss: 0.7183 - val_classification_2_loss: 0.6987 - val_classification_1_acc: 0.5225 - val_classification_2_acc: 0.5150
Epoch 2/150
400/400 [==============================] - 0s 106us/sample - loss: 1.3913 - classification_1_loss: 0.6991 - classification_2_loss: 0.6954 - classification_1_acc: 0.5400 - classification_2_acc: 0.5300 - val_loss: 1.3781 - val_classification_1_loss: 0.6868 - val_classification_2_loss: 0.6929 - val_classification_1_acc: 0.5700 - val_classification_2_acc: 0.5375
Epoch 3/150
400/400 [==============================] - 0s 123us/sample - loss: 1.3560 - classification_1_loss: 0.6664 - classification_2_loss: 0.6893 - classification_1_acc: 0.5925 - classification_2_acc: 0.552

Epoch 23/150
400/400 [==============================] - ETA: 0s - loss: 1.0209 - classification_1_loss: 0.4413 - classification_2_loss: 0.5795 - classification_1_acc: 0.8438 - classification_2_acc: 0.750 - 0s 190us/sample - loss: 1.0429 - classification_1_loss: 0.4265 - classification_2_loss: 0.6215 - classification_1_acc: 0.8650 - classification_2_acc: 0.6900 - val_loss: 1.0555 - val_classification_1_loss: 0.4372 - val_classification_2_loss: 0.6223 - val_classification_1_acc: 0.8600 - val_classification_2_acc: 0.6650
Epoch 24/150
400/400 [==============================] - 0s 132us/sample - loss: 1.0308 - classification_1_loss: 0.4200 - classification_2_loss: 0.6182 - classification_1_acc: 0.8700 - classification_2_acc: 0.6950 - val_loss: 1.0446 - val_classification_1_loss: 0.4231 - val_classification_2_loss: 0.6215 - val_classification_1_acc: 0.8600 - val_classification_2_acc: 0.6575
Epoch 25/150
400/400 [==============================] - 0s 135us/sample - loss: 1.0192 - classificatio

400/400 [==============================] - 0s 111us/sample - loss: 0.8666 - classification_1_loss: 0.2977 - classification_2_loss: 0.5727 - classification_1_acc: 0.8875 - classification_2_acc: 0.7125 - val_loss: 0.8994 - val_classification_1_loss: 0.3192 - val_classification_2_loss: 0.5792 - val_classification_1_acc: 0.8725 - val_classification_2_acc: 0.6950
Epoch 45/150
400/400 [==============================] - 0s 108us/sample - loss: 0.8602 - classification_1_loss: 0.2964 - classification_2_loss: 0.5647 - classification_1_acc: 0.9000 - classification_2_acc: 0.7225 - val_loss: 0.8949 - val_classification_1_loss: 0.3168 - val_classification_2_loss: 0.5815 - val_classification_1_acc: 0.8800 - val_classification_2_acc: 0.6925
Epoch 46/150
400/400 [==============================] - 0s 123us/sample - loss: 0.8548 - classification_1_loss: 0.2892 - classification_2_loss: 0.5697 - classification_1_acc: 0.9025 - classification_2_acc: 0.7075 - val_loss: 0.8903 - val_classification_1_loss: 0.31

Epoch 66/150
400/400 [==============================] - 0s 111us/sample - loss: 0.7668 - classification_1_loss: 0.2290 - classification_2_loss: 0.5349 - classification_1_acc: 0.9125 - classification_2_acc: 0.7500 - val_loss: 0.8193 - val_classification_1_loss: 0.2808 - val_classification_2_loss: 0.5470 - val_classification_1_acc: 0.8875 - val_classification_2_acc: 0.7350
Epoch 67/150
400/400 [==============================] - 0s 188us/sample - loss: 0.7636 - classification_1_loss: 0.2356 - classification_2_loss: 0.5285 - classification_1_acc: 0.9125 - classification_2_acc: 0.7475 - val_loss: 0.8169 - val_classification_1_loss: 0.2786 - val_classification_2_loss: 0.5423 - val_classification_1_acc: 0.8925 - val_classification_2_acc: 0.7350
Epoch 68/150
400/400 [==============================] - 0s 354us/sample - loss: 0.7601 - classification_1_loss: 0.2281 - classification_2_loss: 0.5293 - classification_1_acc: 0.9125 - classification_2_acc: 0.7500 - val_loss: 0.8141 - val_classification

Epoch 88/150
400/400 [==============================] - 0s 241us/sample - loss: 0.7070 - classification_1_loss: 0.1957 - classification_2_loss: 0.5058 - classification_1_acc: 0.9200 - classification_2_acc: 0.7625 - val_loss: 0.7730 - val_classification_1_loss: 0.2519 - val_classification_2_loss: 0.5235 - val_classification_1_acc: 0.8950 - val_classification_2_acc: 0.7375
Epoch 89/150
400/400 [==============================] - 0s 250us/sample - loss: 0.7051 - classification_1_loss: 0.1980 - classification_2_loss: 0.5055 - classification_1_acc: 0.9200 - classification_2_acc: 0.7650 - val_loss: 0.7710 - val_classification_1_loss: 0.2542 - val_classification_2_loss: 0.5269 - val_classification_1_acc: 0.8950 - val_classification_2_acc: 0.7375
Epoch 90/150
400/400 [==============================] - 0s 144us/sample - loss: 0.7031 - classification_1_loss: 0.1942 - classification_2_loss: 0.5095 - classification_1_acc: 0.9225 - classification_2_acc: 0.7600 - val_loss: 0.7693 - val_classification

Epoch 110/150
400/400 [==============================] - 0s 131us/sample - loss: 0.6664 - classification_1_loss: 0.1762 - classification_2_loss: 0.4901 - classification_1_acc: 0.9250 - classification_2_acc: 0.7700 - val_loss: 0.7421 - val_classification_1_loss: 0.2408 - val_classification_2_loss: 0.4942 - val_classification_1_acc: 0.9050 - val_classification_2_acc: 0.7600
Epoch 111/150
400/400 [==============================] - 0s 78us/sample - loss: 0.6643 - classification_1_loss: 0.1781 - classification_2_loss: 0.4857 - classification_1_acc: 0.9275 - classification_2_acc: 0.7725 - val_loss: 0.7392 - val_classification_1_loss: 0.2397 - val_classification_2_loss: 0.5035 - val_classification_1_acc: 0.9075 - val_classification_2_acc: 0.7625
Epoch 112/150
400/400 [==============================] - 0s 95us/sample - loss: 0.6629 - classification_1_loss: 0.1743 - classification_2_loss: 0.4843 - classification_1_acc: 0.9250 - classification_2_acc: 0.7775 - val_loss: 0.7372 - val_classificatio

400/400 [==============================] - 0s 175us/sample - loss: 0.6395 - classification_1_loss: 0.1697 - classification_2_loss: 0.4718 - classification_1_acc: 0.9325 - classification_2_acc: 0.7800 - val_loss: 0.7199 - val_classification_1_loss: 0.2301 - val_classification_2_loss: 0.4903 - val_classification_1_acc: 0.9125 - val_classification_2_acc: 0.7675
Epoch 132/150
400/400 [==============================] - 0s 211us/sample - loss: 0.6390 - classification_1_loss: 0.1650 - classification_2_loss: 0.4677 - classification_1_acc: 0.9300 - classification_2_acc: 0.7800 - val_loss: 0.7202 - val_classification_1_loss: 0.2322 - val_classification_2_loss: 0.4851 - val_classification_1_acc: 0.9100 - val_classification_2_acc: 0.7675
Epoch 133/150
400/400 [==============================] - 0s 127us/sample - loss: 0.6381 - classification_1_loss: 0.1631 - classification_2_loss: 0.4692 - classification_1_acc: 0.9300 - classification_2_acc: 0.7750 - val_loss: 0.7188 - val_classification_1_loss: 0.

**Questions:**
* Does it surprise you that the loss for label 1 is smaller than the loss for label 2?    
* Quick side check... do we overtrain?    

### 5. Custom Layers

Let's define a simple custom layer which is nothing but an affine transformation followed by a relu activation. We need to define:
* \_\_init\_\_()      
* build()   
* call()   
* compute_output_shape()   

In [22]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

class MyStupidDenseLayer(Layer):

    def __init__(self, output_dim, **kwargs):   
        self.output_dim = output_dim
        
        super(MyStupidDenseLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_variable("W",
                                    shape=[int(input_shape[1]),self.output_dim], 
                                   initializer=tf.glorot_normal_initializer(),
                                   trainable=True)
        self.b = self.add_variable("b",
                                    shape=[self.output_dim, ], 
                                   initializer=tf.initializers.zeros(),
                                   trainable=True)
        super(MyStupidDenseLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.nn.relu(tf.tensordot(x, self.W, axes=[[1],[0]]) + self.b)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

Let's use our custom layer in the same architecture. 

In [23]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')
dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model = tf.keras.models.Model(model_input, model_output)
model.compile(loss=losses,  optimizer='adam', metrics=metrics)




In [24]:
model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=149,
    verbose=0
)

model.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 100us/sample - loss: 0.6128 - classification_1_loss: 0.1580 - classification_2_loss: 0.4571 - classification_1_acc: 0.9325 - classification_2_acc: 0.7800 - val_loss: 0.6883 - val_classification_1_loss: 0.2224 - val_classification_2_loss: 0.4668 - val_classification_1_acc: 0.9125 - val_classification_2_acc: 0.7825


Close enough. So the custom layer works exactly as expected.


### 6. Sharing & Freezing Weights 


Let's define an identical model. The weights for this model will be re-initialized, so upon training for 1 epoch the loss will be much higher than after 150 epochs for the original model: 

In [25]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')
dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')


classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model_mirror = tf.keras.models.Model(model_input, model_output)
model_mirror.compile(loss=losses,  optimizer='adam', metrics=metrics)




In [26]:
model_mirror.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 1s 1ms/sample - loss: 1.4418 - classification_1_loss: 0.7707 - classification_2_loss: 0.6700 - classification_1_acc: 0.3250 - classification_2_acc: 0.5900 - val_loss: 1.4239 - val_classification_1_loss: 0.7571 - val_classification_2_loss: 0.6641 - val_classification_1_acc: 0.3525 - val_classification_2_acc: 0.6300


Indeed, this model would make very different predictions - the loss is ~1.4 vs ~0.6. No surprise, obviously, as this model has not been trained.

What if we manually set the the weights of the second model to the weights of the first? Let's look at the layers for both models and then copy the weights:

In [27]:
model.layers

Let's look at one layer in more detail:

In [28]:
len(model.layers[1].get_weights())

2

Why would that be? No magic.. weights and biases of layer!

In [29]:
np.shape(model.layers[1].get_weights()[0])

(10, 10)

In [30]:
np.shape(model.layers[1].get_weights()[1])

(10,)

Cool. What is the name of the layer?

In [31]:
model.layers[1].name

'dense_1'

Correct. Is it trainable?

In [32]:
model.layers[1].trainable

True

Great. Are the layers of the second model essentially the same?

In [33]:
model_mirror.layers

Yup. (We could look at shapes, etc, but won't do that now.) Now we set the second model's weights to equal the first model's trained weights:

In [34]:
model_mirror.layers[1].set_weights(model.layers[1].get_weights())
model_mirror.layers[2].set_weights(model.layers[2].get_weights())
model_mirror.layers[3].set_weights(model.layers[3].get_weights())
model_mirror.layers[4].set_weights(model.layers[4].get_weights())

**Question:** Why don't we set the weights for layer 0?

Okay. What is the loss now for the mirror model? 

In [35]:
model_mirror.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 248us/sample - loss: 0.6146 - classification_1_loss: 0.1575 - classification_2_loss: 0.4642 - classification_1_acc: 0.9300 - classification_2_acc: 0.7825 - val_loss: 0.6874 - val_classification_1_loss: 0.2141 - val_classification_2_loss: 0.4658 - val_classification_1_acc: 0.9125 - val_classification_2_acc: 0.7825


Cool! Essentially the same as for the original one! (Not a surprise actually... but great that it works.)

Let's now turn to **freezing** layers. For example, you may want to copy weights from another model but hold those weights fixed upon further training. To test this, we will do a simple toy exercise: freeze a layer of the second model and train it further. Let's compare some weights before and after training. 

We start by defining a new model that again has the same architecture, **but we set the trainable-parameter for one layer to False**: 

In [37]:
# Define Input layer(s)
input_numbers = tf.keras.layers.Input(shape=(10,), name="input_numbers")

# Define separate hidden layers and acting on (same!) input
dense_layer_1 = MyStupidDenseLayer(10, name='dense_1')

dense_layer_2 = MyStupidDenseLayer(10, name='dense_2')
dense_layer_2.trainable = False                                   # Freeze this layer

dense_output_1 = dense_layer_1(input_numbers)
dense_output_2 = dense_layer_2(input_numbers)



# Define classification layer and act on previous output to classify examples
classification_layer_1 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_1')

classification_layer_2 = tf.keras.layers.Dense(1, 
                            activation='sigmoid', name='classification_2')



classification_output_1 = classification_layer_1(dense_output_1)
classification_output_2 = classification_layer_2(dense_output_2)



# Build and compile model

model_input = input_numbers
model_output = [classification_output_1, classification_output_2]
losses = ['binary_crossentropy', 'binary_crossentropy']
metrics = ['acc', 'acc']

model_mirror_2 = tf.keras.models.Model(model_input, model_output)
model_mirror_2.compile(loss=losses,  optimizer='adam', metrics=metrics)


Next, we again set the weights to mirror the first model:

In [38]:
model_mirror_2.layers[1].set_weights(model.layers[1].get_weights())
model_mirror_2.layers[2].set_weights(model.layers[2].get_weights())
model_mirror_2.layers[3].set_weights(model.layers[3].get_weights())
model_mirror_2.layers[4].set_weights(model.layers[4].get_weights())

What are (some of) the weights now, before we train further?

In [39]:
model_mirror_2.layers[1].get_weights()[0][:3, :3]

array([[ 1.0916228 ,  0.0080574 , -0.6700347 ],
       [ 0.36431167, -0.7220702 , -0.32361656],
       [ 0.32802176,  0.30155778, -0.45293397]], dtype=float32)

In [40]:
model_mirror_2.layers[2].get_weights()[0][:3, :3]

array([[ 0.59885055, -0.04448628, -0.04327861],
       [-0.55413663,  0.1483072 ,  0.22182831],
       [-0.03545466,  0.5287597 , -0.05535174]], dtype=float32)

We will compare that with the values post further training. Let's also check the 'trainable' settings:

In [41]:
model_mirror_2.layers[1].trainable

True

In [42]:
model_mirror_2.layers[2].trainable

False

Good, so it shows that one hidden layer supposedly is trainable, the other one is not. Is that true? We train more and compare weights compared to what they were:

In [43]:
model_mirror_2.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=40,
    verbose=0
)
model_mirror_2.fit(
    train_X,
    [train_a_y, train_b_y],
    validation_data=[test_X, [test_a_y, test_b_y]],
    epochs=1,
    verbose=1
)

Train on 400 samples, validate on 400 samples
400/400 [==============================] - 0s 84us/sample - loss: 0.5971 - classification_1_loss: 0.1468 - classification_2_loss: 0.4534 - classification_1_acc: 0.9300 - classification_2_acc: 0.7800 - val_loss: 0.6748 - val_classification_1_loss: 0.2097 - val_classification_2_loss: 0.4654 - val_classification_1_acc: 0.9250 - val_classification_2_acc: 0.7900


In [44]:
model_mirror_2.layers[1].get_weights()[0][:3, :3]

array([[ 1.1080302 , -0.01284085, -0.70266557],
       [ 0.35729578, -0.7506223 , -0.33273327],
       [ 0.36315778,  0.3460176 , -0.4623005 ]], dtype=float32)

These weights **did change**.

In [45]:
model_mirror_2.layers[2].get_weights()[0][:3, :3]

array([[ 0.59885055, -0.04448628, -0.04327861],
       [-0.55413663,  0.1483072 ,  0.22182831],
       [-0.03545466,  0.5287597 , -0.05535174]], dtype=float32)

These weights **did not change**. So here we could see layer freezing at work!